In [1]:
# Imports
%load_ext autoreload
%autoreload 2 
# !apt-get install -y xvfb
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
from tqdm.notebook import tqdm

In [2]:
# Load the e5 csv and add weibel gen (if uncommented)
df_orig = pd.read_csv(os.path.abspath("/home/sneha/e5lungairwaysvida_20140211.csv"))

/tmp/ipykernel_28655/1085606535.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_orig = pd.read_csv(os.path.abspath("/home/sneha/e5lungairwaysvida_20140211.csv"))


In [2]:
# add weibel
def _get_weibel_gen(participant_df):
    # groupby idno
    participant_df['new_weibel_gen'] =0
    groups = participant_df.groupby('idno')
    for name, group in tqdm(groups, desc='Iterating ids'):
#         print(name, group)
        start_end =  group[['startbpid', 'endbpid']].values
        keys_ = np.unique(start_end, return_counts=False)
        weibel_gen = dict.fromkeys(keys_, 0)
        weibel_gen[-1]=-1
    #     print(weibel_gen)
        for i in range(len(keys_)):
            key_test=keys_[i]
#             print('looking at key', key_test)
            # find children of this key
            children = group.loc[participant_df.startbpid==key_test]['endbpid'].values
#             print('children of key', children)
            for child in children:
                weibel_gen[child] = weibel_gen[key_test]+1
        participant_df.loc[participant_df.idno == name, 'new_weibel_gen'] = participant_df.loc[participant_df.idno == name, 'endbpid'].copy().map(weibel_gen)
    return participant_df

add_weibel= False

if add_weibel:
    df_weibel = _get_weibel_gen(df_orig.copy())
    df_weibel.rename(columns={'new_weibel_gen':'weibel_generation'}, inplace=True)
    df_weibel.to_csv(os.path.abspath('/home/sneha/e5lungairwaysvida_20140211_weibel.csv'), index=False)
else:
    df_weibel = pd.read_csv(os.path.abspath('/home/sneha/e5lungairwaysvida_20140211_weibel.csv'))
 

/tmp/ipykernel_9248/3220799132.py:31: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_weibel = pd.read_csv(os.path.abspath('/home/sneha/e5lungairwaysvida_20140211_weibel.csv'))


In [3]:
# Drop NaNs in start & end id
print(df_weibel.idno.nunique(), len(df_weibel))
df_weibel.dropna(subset=['startbpid', 'endbpid'], inplace=True)
print(df_weibel.idno.nunique(), len(df_weibel))

3194 992300
2998 992104


In [4]:
# groupby idno
def _remove_orphans(df):
    orphan_check = np.zeros_like(df.endbpid.values, dtype=bool)
    groups = df.groupby('idno')
    i = 0
    for name, group in tqdm(groups, desc='Iterating ids'):
        orphan_check[i:i+len(group)] =\
        np.isin(group.startbpid.values, group.endbpid.values) | \
        (group.startbpid == -1)
        i = i+len(group)
    return orphan_check

orphan_check = _remove_orphans(df_weibel)
# np_list = np.array([1,2,3])
# test = np.array([1,5,4,3])
# np.isin(test, np_list)
df_no_orphans = df_weibel.loc[orphan_check]# .loc[df_weibel.idno.isin([3010007,3010058])]

Iterating ids:   0%|          | 0/2998 [00:00<?, ?it/s]

In [5]:
# 196 ids have only NaN start + End IDs --> these were inadvertently removed by the orphans --> should remove before
print("Orphan branches removed:", len(df_weibel)-len(df_no_orphans))


Orphan branches removed: 84


In [6]:
# Count trifurcations
num_children = df_no_orphans.groupby(
    [pd.Grouper('idno'), pd.Grouper('startbpid')]
).agg({'endbpid':"count",
      'weibel_generation': min}).reset_index()

not_bifurcation = num_children.loc[(num_children.startbpid!=-1) & (num_children.endbpid !=2)].sort_values(by='endbpid')
not_bifurcation

,idno,startbpid,endbpid,weibel_generation
396040,7019190,61.0,1,7
423352,8011290,273.0,1,8
151713,4016491,10.0,1,4
333891,7010931,155.0,1,8
180097,4021010,230.0,1,11
...,...,...,...,...
180011,4021010,8.0,7,2
406730,7041470,24.0,7,5
173351,4019288,98.0,8,6
71149,3018652,101.0,9,7


In [7]:
# one startbpid has 11 end points.
df_no_orphans.loc[(df_no_orphans.idno==4019288) & (df_no_orphans.startbpid==28)][['idno', 'startbpid', 'endbpid', 'centerlinelength', 'avginnerarea']]

,idno,startbpid,endbpid,centerlinelength,avginnerarea
350073,4019288,28.0,51.0,1.289060,-1.0000
350074,4019288,28.0,52.0,2.080170,51.8231
350075,4019288,28.0,53.0,0.815733,-1.0000
350076,4019288,28.0,97.0,2.839390,55.0024
350077,4019288,28.0,96.0,15.942400,14.2219
350078,4019288,28.0,98.0,1.890330,-1.0000
350079,4019288,28.0,99.0,2.038190,-1.0000
350080,4019288,28.0,249.0,4.798020,58.6123
350081,4019288,28.0,250.0,2.765950,-1.0000
350082,4019288,28.0,164.0,3.506730,41.8836


In [14]:
# how many 'xfurcations'
x_furcations = num_children.groupby('endbpid').agg(
    {'startbpid':'count', 'weibel_generation':"mean"}
).reset_index().rename(columns={'startbpid':'count', 'endbpid':'x-furcation','weibel_generation':'mean_weibel_gen' })

print(f"{x_furcations.loc[x_furcations['x-furcation']!=2]['count'].sum()} branch points are not bifurcations")
x_furcations

15873 branch points are not bifurcations


,x-furcation,count,mean_weibel_gen
0,1,3053,0.179823
1,2,475058,7.368273
2,3,12502,6.986402
3,4,273,7.128205
4,5,30,6.900000
5,6,9,6.333333
6,7,3,4.000000
7,8,1,6.000000
8,9,1,7.000000
9,11,1,5.000000


In [25]:
# remove non-bifurcations for now
df_2 = pd.DataFrame({'to_remove': (num_children.startbpid!=-1) & (num_children.endbpid !=2)})

# not bifurcation df
start_bpids_to_remove = pd.concat([num_children, df_2], axis=1)

# merge onto full df
df_no_orphans = df_no_orphans.merge(start_bpids_to_remove[['idno', "startbpid", 'to_remove']], on=['idno', "startbpid"], how='left')
df_no_orphans_only_bi = df_no_orphans.loc[df_no_orphans.to_remove==False]
print(f"Removing non-birfucations (excluding trachea which only has one endpt -1 to 1) = removing {-(len(df_no_orphans_only_bi)-len(df_no_orphans))} rows")
print(f"Df now has {len(df_no_orphans_only_bi)} rows and {df_no_orphans_only_bi.idno.nunique()} participants")

Removing non-birfucations (excluding trachea which only has one endpt -1 to 1) = removing -38920 rows
Df now has 953100 rows and 2998 participants


In [26]:
check = df_no_orphans_only_bi.groupby([pd.Grouper('idno'), pd.Grouper('startbpid')]).agg({"endbpid":"count"}).sort_values(by="endbpid").reset_index()
display(check)
# only bifurcations, and trachea has 1 end pt only: works
check.loc[(check.startbpid != -1) & (check.endbpid == 1)]

,idno,startbpid,endbpid
0,3010007,-1.0,1
1,3021467,-1.0,1
2,7040792,-1.0,1
3,3021459,-1.0,1
4,7040822,-1.0,1
...,...,...,...
478037,4018141,181.0,2
478038,4018141,172.0,2
478039,4018141,171.0,2
478040,4018141,214.0,2


,idno,startbpid,endbpid


In [27]:
# Count scan problem
values, counts = np.unique(df_no_orphans_only_bi.scan_problem.values, return_counts=True)
print("Scan problem value: frequency ==>",dict(zip(values, counts)))
print(f"Number of rows with scan_problem {(df_no_orphans_only_bi.scan_problem !=0).sum()} =={100*(df_no_orphans_only_bi.scan_problem !=0).sum()/len(df_no_orphans_only_bi):.2f}% of rows in df")
print(f"Number of unique participants impacted by scan problems: {df_no_orphans_only_bi.loc[df_no_orphans_only_bi.scan_problem!=0].idno.nunique()} == {100*df_no_orphans_only_bi.loc[df_no_orphans_only_bi.scan_problem!=0].idno.nunique()/ df_no_orphans_only_bi.idno.nunique():.2f}% of participants")


Scan problem value: frequency ==> {0: 926841, 1: 1148, 2: 2408, 3: 19674, 4: 3029}
Number of rows with scan_problem 26259 ==2.76% of rows in df
Number of unique participants impacted by scan problems: 87 == 2.90% of participants


In [28]:
test = df_no_orphans_only_bi.loc[df_no_orphans_only_bi.idno.isin(df_no_orphans_only_bi.loc[df_no_orphans_only_bi.scan_problem!=0].idno.unique())].groupby('idno').agg({'idno': "count", 'scan_problem':[min, 'mean', max]})
test[test[('scan_problem', 'min')]!=test[('scan_problem', 'max')] ] # none - all of a participant has same scan problem --> drop these participants for now

Empty DataFrame
Columns: [(idno, count), (scan_problem, min), (scan_problem, mean), (scan_problem, max)]
Index: []

In [31]:
# remove 87 participants with marked scan problems
df_no_scan_prob_no_orphans_only_bi = df_no_orphans_only_bi.loc[~(df_no_orphans_only_bi.idno.isin(df_no_orphans_only_bi.loc[df_no_orphans_only_bi.scan_problem!=0].idno.unique()))]

print(df_no_orphans_only_bi.idno.nunique(), df_no_scan_prob_no_orphans_only_bi.idno.nunique())

2998 2911


In [32]:
# check units: units are in mm / mm^2
print(df_no_scan_prob_no_orphans_only_bi.loc[0, ["idno", "startbpid", "endbpid", "centerlinelength", "avginnerarea", "avginnerequivalentcirclediamete"]])
" trachea diameter", np.sqrt(351.513/np.pi)*2

idno                               3010007
startbpid                             -1.0
endbpid                                1.0
centerlinelength                   104.962
avginnerarea                       351.513
avginnerequivalentcirclediamete    21.3829
Name: 0, dtype: object


(' trachea diameter', 21.155619870107593)

In [33]:
# rows w -1s --> below shows missing angles have mean centerline length of 1.87 vs full dataset of 10.8 so v sort segments w small CSA (2.9 vs 11.8)
missing_angles = df_no_scan_prob_no_orphans_only_bi.loc[df_no_scan_prob_no_orphans_only_bi.dircosx == -1][['startbpid', 'endbpid','angle','dircosx','dircosy','dircosz', 'weibel_generation', 'centerlinelength', 'avginnerarea']]
print('Missing Angles')
display(missing_angles.describe().loc[["mean", "std"],:].round(3))
print('Whole df')
display(df_no_scan_prob_no_orphans_only_bi[['startbpid', 'endbpid','angle','dircosx','dircosy','dircosz', 'weibel_generation', 'centerlinelength', 'avginnerarea']].describe().loc[["mean", "std"],:].round(3))

Missing Angles


,startbpid,endbpid,angle,dircosx,dircosy,dircosz,weibel_generation,centerlinelength,avginnerarea
mean,203.514,255.061,115.679,-1.0,0.0,0.0,8.439,1.881,2.810
std,128.588,132.256,25.652,0.0,0.0,0.0,1.924,2.341,11.864


Whole df


,startbpid,endbpid,angle,dircosx,dircosy,dircosz,weibel_generation,centerlinelength,avginnerarea
mean,152.524,200.119,143.740,-0.016,0.049,0.136,7.348,10.878,11.667
std,127.452,137.944,24.081,0.545,0.544,0.621,2.298,9.944,26.005


In [34]:
# chevk the parents to see if the missing vectors are ok
#df_no_orphans.loc[(df_no_orphans.dircosx == -1][['idno','startbpid', 'endbpid']] # 	3010007, 259 parent, 317 child
parent = df_no_scan_prob_no_orphans_only_bi.loc[(df_no_scan_prob_no_orphans_only_bi.idno==3010007)& (df_no_scan_prob_no_orphans_only_bi.endbpid==259)][['idno','startbpid', 'endbpid', 'angle', 'centerlinelength', 'dircosx', 'dircosy', 'dircosz']]
child = df_no_scan_prob_no_orphans_only_bi.loc[(df_no_scan_prob_no_orphans_only_bi.idno==3010007)& (df_no_scan_prob_no_orphans_only_bi.endbpid==317)][['idno','startbpid', 'endbpid', 'angle', 'centerlinelength', 'dircosx', 'dircosy', 'dircosz']]
display(parent, child)

v1 = parent[['dircosx', 'dircosy', 'dircosz']].values
v2 = child[['dircosx', 'dircosy', 'dircosz']].values
print(180-(np.arccos(np.dot(v1.squeeze(), v2.squeeze()))*180/np.pi))

np.arccos(v2)*180/np.pi

,idno,startbpid,endbpid,angle,centerlinelength,dircosx,dircosy,dircosz
257,3010007,186.0,259.0,146.425,14.6101,-0.965609,-0.241402,0.096561


,idno,startbpid,endbpid,angle,centerlinelength,dircosx,dircosy,dircosz
315,3010007,259.0,317.0,164.93,9.1914,-1.0,0.0,0.0


164.93002239617675


array([[180.,  90.,  90.]])

In [35]:
# crosstab lobes w names + sublobes w named branches (remove)
df_no_scan_prob_no_orphans_only_bi.loc[(df_no_scan_prob_no_orphans_only_bi.idno==df_no_scan_prob_no_orphans_only_bi.idno.values[0]) & (df_no_scan_prob_no_orphans_only_bi.anatomicalname != "-")][['idno','startbpid', 'endbpid', 'anatomicalname', 'weibel_generation', 'lobe', 'sublobe']]

,idno,startbpid,endbpid,anatomicalname,weibel_generation,lobe,sublobe
0,3010007,-1.0,1.0,Trachea,0,0.0,0.0
1,3010007,1.0,3.0,LMB,1,0.0,0.0
2,3010007,1.0,2.0,RMB,1,0.0,0.0
3,3010007,2.0,4.0,BronInt,2,0.0,0.0
4,3010007,2.0,5.0,RUL,2,3.0,0.0
5,3010007,3.0,6.0,LLB6,2,2.0,0.0
6,3010007,3.0,7.0,LUL,2,1.0,0.0
7,3010007,4.0,9.0,RB4+5,3,4.0,0.0
9,3010007,5.0,10.0,RB1,3,3.0,10.0
11,3010007,6.0,13.0,LB6,3,2.0,6.0


In [36]:
useful_cols = ['idno',
 'anatomicalname',
 'centerlinelength','avginnerarea','lobe',
 'sublobe',
 'endbpid',
 'startbpid',
 'angle',
 'dircosx',
 'dircosy',
 'dircosz', 'weibel_generation']

In [ ]:
# save down cleaned file
df_cleaned = df_no_scan_prob_no_orphans_only_bi[useful_cols]
df_cleaned.to_csv('e5_cleaned_v1.csv', index=False)